<a href="https://colab.research.google.com/github/anitha67/100DaysofMLCode/blob/master/test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Text Classification Using RNN (Qualitative or Quantitative)

In [17]:
import numpy as np

import tensorflow_datasets as tfds
import tensorflow as tf

tfds.disable_progress_bar()

In [19]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [20]:
import pandas as pd
df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/test_data/bertbaseuncased/cases_worknote.csv',encoding="latin1" )

In [21]:
df.head()

,id,user,note,timestamp,is_first_response,case_id,is_qualitative
0,216,"{""id"": 1, ""email"": ""system@localhost"", ""userna...",<p>Check URL</p><p><br></p><p>Inputs:</p><p><b...,2022-01-22 05:02:46.545945+00,False,52,True
1,217,"{""id"": 1, ""email"": ""system@localhost"", ""userna...",<p>http://ugandakidneyfoundation.org/wp-conten...,2022-01-22 05:02:47.084561+00,False,52,True
2,218,"{""id"": 1, ""email"": ""system@localhost"", ""userna...",<p>Get URl scan report from VirusTotal</p><p><...,2022-01-22 05:02:48.526829+00,False,52,True
3,219,"{""id"": 1, ""email"": ""system@localhost"", ""userna...",<p>Check URL</p><p><br></p><p>Inputs:</p><p><b...,2022-01-22 05:04:30.883732+00,False,53,True
4,220,"{""id"": 1, ""email"": ""system@localhost"", ""userna...",<p>https://www.red-by-sfr.fr/info-clients/evol...,2022-01-22 05:04:31.410755+00,False,53,False


In [22]:
import re
import spacy
def regex_sub(text):
    ip_pattern = r'((((([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])\.){3}([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5]))([^0-9]|$))|(((([0-9A-Fa-f]{1,4}:){7}([0-9A-Fa-f]{1,4}|:))|(([0-9A-Fa-f]{1,4}:){6}(:[0-9A-Fa-f]{1,4}|((25[0-5]|2[0-4]\d|1\d\d|[1-9]?\d)(\.(25[0-5]|2[0-4]\d|1\d\d|[1-9]?\d)){3})|:))|(([0-9A-Fa-f]{1,4}:){5}(((:[0-9A-Fa-f]{1,4}){1,2})|:((25[0-5]|2[0-4]\d|1\d\d|[1-9]?\d)(\.(25[0-5]|2[0-4]\d|1\d\d|[1-9]?\d)){3})|:))|(([0-9A-Fa-f]{1,4}:){4}(((:[0-9A-Fa-f]{1,4}){1,3})|((:[0-9A-Fa-f]{1,4})?:((25[0-5]|2[0-4]\d|1\d\d|[1-9]?\d)(\.(25[0-5]|2[0-4]\d|1\d\d|[1-9]?\d)){3}))|:))|(([0-9A-Fa-f]{1,4}:){3}(((:[0-9A-Fa-f]{1,4}){1,4})|((:[0-9A-Fa-f]{1,4}){0,2}:((25[0-5]|2[0-4]\d|1\d\d|[1-9]?\d)(\.(25[0-5]|2[0-4]\d|1\d\d|[1-9]?\d)){3}))|:))|(([0-9A-Fa-f]{1,4}:){2}(((:[0-9A-Fa-f]{1,4}){1,5})|((:[0-9A-Fa-f]{1,4}){0,3}:((25[0-5]|2[0-4]\d|1\d\d|[1-9]?\d)(\.(25[0-5]|2[0-4]\d|1\d\d|[1-9]?\d)){3}))|:))|(([0-9A-Fa-f]{1,4}:){1}(((:[0-9A-Fa-f]{1,4}){1,6})|((:[0-9A-Fa-f]{1,4}){0,4}:((25[0-5]|2[0-4]\d|1\d\d|[1-9]?\d)(\.(25[0-5]|2[0-4]\d|1\d\d|[1-9]?\d)){3}))|:))|(:(((:[0-9A-Fa-f]{1,4}){1,7})|((:[0-9A-Fa-f]{1,4}){0,5}:((25[0-5]|2[0-4]\d|1\d\d|[1-9]?\d)(\.(25[0-5]|2[0-4]\d|1\d\d|[1-9]?\d)){3}))|:)))(%.+)?([^A-Za-z0-9]|$)))'
    yyyy_date_pattern = r'([1-2][0-9]{3}[-\.\/])((([1-9]|1[0-2]|0[1-9])[-\.\/]([1-9]|0[1-9]|1[0-9]|2[0-9]|3[0-1])[^0-9])|(([1-9]|0[1-9]|1[0-9]|2[0-9]|3[0-1])[-\.\/]([1-9]|0[1-9]|1[0-2])[^0-9]))'
    m_d_y_data_pattern = r'[^0-9]((([1-9]|0[1-9]|1[0-2])[-\.\/]([1-9]|0[1-9]|1[0-9]|2[0-9]|3[0-1]))|(([1-9]|0[1-9]|1[0-9]|2[0-9]|3[0-1])[-\.\/]([1-9]|1[0-2]|0[1-9])))[-\.\/]([1-2][0-9]{3}|[0-9]{2})'
    timestamp_pattern_1 = r'([1-2][0-9]{3}[-\.\/])((([1-9]|1[0-2]|0[1-9])[-\.\/]([1-9]|0[1-9]|1[0-9]|2[0-9]|3[0-1])[^0-9])|(([1-9]|0[1-9]|1[0-9]|2[0-9]|3[0-1])[-\.\/]([1-9]|0[1-9]|1[0-2])))[\s]([0-9]|[0-1][0-9]|2[0-3]):([0-9]|[0-5][0-9]):([0-9]|[0-5][0-9])[^0-9]'
    timestamp_pattern_2 = r'[^0-9]((([1-9]|0[1-9]|1[0-2])[-\.\/]([1-9]|0[1-9]|1[0-9]|2[0-9]|3[0-1]))|(([1-9]|0[1-9]|1[0-9]|2[0-9]|3[0-1])[-\.\/]([1-9]|1[0-2]|0[1-9])))[-\.\/]([1-2][0-9]{3}|[0-9]{2})[\s]([0-9]|[0-1][0-9]|2[0-3]):([0-9]|[0-5][0-9]):([0-9]|[0-5][0-9])[^0-9]'
    windows_file_path = r'[A-Za-z]:[\\][^\s]*'
    unix_file_path = r''
    url_pattern = r'http[s]?:[\/]{2}[^\s]+'
    email_pattern = r'[\S]+@[^\s\.]+\.[\S]+'
    html_tag = r'\<[^\>]*\>'
    text = re.sub(html_tag,' ',text)
    text = re.sub(windows_file_path,' filepath ',text)
    text = re.sub(url_pattern,' url ',text)
    text = re.sub(email_pattern,' email ',text)
    text = re.sub(ip_pattern,' ipaddress ',text)
    text = re.sub(timestamp_pattern_1,' timestamp ',text)
    text = re.sub(timestamp_pattern_2,' timestamp ',text)
    text = re.sub(yyyy_date_pattern,' timestamp ',text)
    text = re.sub(m_d_y_data_pattern,' timestamp ',text)
    text = text.lower()
    text = re.sub('[^a-z0-9\s]',' ',text)
#     text = nlp_oper(text)
    return text

In [23]:
df['note'] = df['note'].apply(lambda x:regex_sub(x))
df.head()

,id,user,note,timestamp,is_first_response,case_id,is_qualitative
0,216,"{""id"": 1, ""email"": ""system@localhost"", ""userna...",check url inputs url url o...,2022-01-22 05:02:46.545945+00,False,52,True
1,217,"{""id"": 1, ""email"": ""system@localhost"", ""userna...",url was not found on phishtank,2022-01-22 05:02:47.084561+00,False,52,True
2,218,"{""id"": 1, ""email"": ""system@localhost"", ""userna...",get url scan report from virustotal input...,2022-01-22 05:02:48.526829+00,False,52,True
3,219,"{""id"": 1, ""email"": ""system@localhost"", ""userna...",check url inputs url url o...,2022-01-22 05:04:30.883732+00,False,53,True
4,220,"{""id"": 1, ""email"": ""system@localhost"", ""userna...",url was found on phishtank,2022-01-22 05:04:31.410755+00,False,53,False


In [56]:
def user(string):
  pattern =r'[A-Za-z]+(?=\d{0,}@)'
  username= re.findall(pattern,string)
  #print('username is :',username)
  return username

In [58]:
df['user1'] = df['user'].apply(lambda x:user(x))
df.head()




,id,user,note,timestamp,is_first_response,case_id,is_qualitative,username,user1
0,216,"{""id"": 1, ""email"": ""system@localhost"", ""userna...",check url inputs url url o...,2022-01-22 05:02:46.545945+00,False,52,True,"{""id"": 1, ""email"": ""system",[system]
1,217,"{""id"": 1, ""email"": ""system@localhost"", ""userna...",url was not found on phishtank,2022-01-22 05:02:47.084561+00,False,52,True,"{""id"": 1, ""email"": ""system",[system]
2,218,"{""id"": 1, ""email"": ""system@localhost"", ""userna...",get url scan report from virustotal input...,2022-01-22 05:02:48.526829+00,False,52,True,"{""id"": 1, ""email"": ""system",[system]
3,219,"{""id"": 1, ""email"": ""system@localhost"", ""userna...",check url inputs url url o...,2022-01-22 05:04:30.883732+00,False,53,True,"{""id"": 1, ""email"": ""system",[system]
4,220,"{""id"": 1, ""email"": ""system@localhost"", ""userna...",url was found on phishtank,2022-01-22 05:04:31.410755+00,False,53,False,"{""id"": 1, ""email"": ""system",[system]


In [71]:
def username(string):
  for (user1,item) in df.iteritems():
    if item.values == 'system':
      print("username is: ",item.values)
    else:
        print("username is : ",item.values)
  return string 
  

In [72]:
df['user2'] = df['user1'].apply(lambda x:username(x))
df['user2']

username is :  [ 216  217  218 ... 1568 1569 1570]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  This is separate from the ipykernel package so we can avoid doing imports until


ValueError: ignored